In [1]:
from starlette.applications import Starlette
from starlette.responses import HTMLResponse, JSONResponse
from starlette.staticfiles import StaticFiles
import sys
sys.path.append('../recommender/')
from starlette.middleware.cors import CORSMiddleware
import uvicorn, aiohttp, asyncio
from io import BytesIO
from pathlib import Path
from cloth_detector import *
from style_recommender import *

In [ ]:
app = Starlette()
app.add_middleware(CORSMiddleware, allow_origins=['*'], allow_headers=['X-Requested-With', 'Content-Type'])
app.mount('/static', StaticFiles(directory='../static'))
app.mount('/static', StaticFiles(directory='../static/image_data'))
app.mount('/static', StaticFiles(directory='../static/tmp'))


@app.route('/')
def index(request):
    html = Path("../view/index.html")
    return HTMLResponse(html.open().read())

@app.route('/email', methods=['POST'])
async def send_email(request):
    json = await request.json()
    print(json)
    return JSONResponse({})

@app.route('/analyze', methods=['POST'])
async def detect_items(request):
    json = await request.json()
    result = get_recommendations(json)
    print(result)
    return JSONResponse({"results":result})

@app.route('/detect', methods=['POST'])
async def analyze(request):
    data = await request.form()
    img_bytes = await (data['file'].read())
    response = get_detected_items(Image.open(BytesIO(img_bytes)))
    print(response)
    return JSONResponse(response)

if __name__ == '__main__':
    uvicorn.run(app=app, host='0.0.0.0', port=1997)

INFO: Started server process [4477]
INFO: Waiting for application startup.
INFO: Uvicorn running on http://0.0.0.0:1997 (Press CTRL+C to quit)
INFO: ('130.159.236.1', 46064) - "GET / HTTP/1.1" 200
INFO: ('130.159.236.1', 46064) - "GET /static/style.css HTTP/1.1" 304
INFO: ('130.159.236.1', 46066) - "GET /static/front-photo.jpg HTTP/1.1" 304
/home/vanio/anaconda3/envs/fastai/lib/python3.6/site-packages/PIL/Image.py:2655: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)
INFO: ('130.159.236.1', 46066) - "POST /detect HTTP/1.1" 500
ERROR: Exception in ASGI application
Traceback (most recent call last):
  File "/home/vanio/anaconda3/envs/fastai/lib/python3.6/site-packages/uvicorn/protocols/http/httptools_impl.py", line 372, in run_asgi
    result = await asgi(self.receive, self.send)
  File "/home/vanio/anaconda3/envs/fastai/lib/python3.6/site-packages/starlette/middleware/errors.py", line 125, in asgi
    raise exc from None
  File

{'cropped_items': ['../static/tmp/70a4b85e-86f1-4ba9-afdd-f2df1250b238.jpg', '../static/tmp/603e1ba7-6e3b-498f-b3c7-7f6e3993ca8e.jpg', '../static/tmp/e6e21393-1ad2-4a86-96b8-64827ac2dbe5.jpg'], 'categories': [0, 7, 8]}


INFO: ('130.159.236.1', 46164) - "GET /static/style.css HTTP/1.1" 304
INFO: ('130.159.236.1', 46166) - "GET /static/front-photo.jpg HTTP/1.1" 304
INFO: ('130.159.236.1', 46176) - "POST /detect HTTP/1.1" 200
INFO: ('130.159.236.1', 46176) - "GET /static/tmp/65d05976-0e1c-40ee-8196-ae6333803f50.jpg HTTP/1.1" 200
INFO: ('130.159.236.1', 46180) - "GET /static/tmp/dcee715a-5e4d-4e44-b4d2-adece469754e.jpg HTTP/1.1" 200
INFO: ('130.159.236.1', 46178) - "GET /static/tmp/bf4d3400-e2a7-4a67-9218-65ab5661320a.jpg HTTP/1.1" 200


{'cropped_items': ['../static/tmp/65d05976-0e1c-40ee-8196-ae6333803f50.jpg', '../static/tmp/dcee715a-5e4d-4e44-b4d2-adece469754e.jpg', '../static/tmp/bf4d3400-e2a7-4a67-9218-65ab5661320a.jpg'], 'categories': [0, 7, 8]}


INFO: ('130.159.236.1', 46188) - "POST /detect HTTP/1.1" 200
INFO: ('130.159.236.1', 46188) - "GET /static/tmp/819a279c-cc46-4984-b5cb-f15c45eff37e.jpg HTTP/1.1" 200
INFO: ('130.159.236.1', 46196) - "GET /static/tmp/7eca51cd-2390-4569-8939-409474a00ac4.jpg HTTP/1.1" 200
INFO: ('130.159.236.1', 46198) - "GET /static/tmp/410eec99-44fe-4bb7-8d8b-6b8bfb08e7b3.jpg HTTP/1.1" 200
INFO: ('130.159.236.1', 46190) - "GET /static/tmp/401ddcd5-15f8-490c-b3cf-955b7d25c75b.jpg HTTP/1.1" 200
INFO: ('130.159.236.1', 46194) - "GET /static/tmp/0d491d76-cda6-4f22-b25b-a8b228757808.jpg HTTP/1.1" 200
INFO: ('130.159.236.1', 46192) - "GET /static/tmp/49668b45-9820-4314-8f48-40b8ac59a782.jpg HTTP/1.1" 200


...

{'name': 'sad', 'email': 'asd', 'comments': 'asd'}


INFO: ('130.159.236.1', 52098) - "GET / HTTP/1.1" 200
INFO: ('82.28.48.184', 50815) - "GET / HTTP/1.1" 200
INFO: ('82.28.48.184', 50816) - "GET /static/image-picker.css HTTP/1.1" 200
INFO: ('82.28.48.184', 50815) - "GET /static/style.css HTTP/1.1" 200
INFO: ('82.28.48.184', 50819) - "GET /static/example.jpg HTTP/1.1" 200
INFO: ('82.28.48.184', 50815) - "GET /static/image-picker.min.js HTTP/1.1" 200
INFO: ('82.28.48.184', 50816) - "GET /static/client.js HTTP/1.1" 200
INFO: ('82.28.48.184', 50819) - "GET /static/scroll-reveal-conf.js HTTP/1.1" 200
INFO: ('82.28.48.184', 50823) - "GET /static/example-cropped2.jpg HTTP/1.1" 200
INFO: ('82.28.48.184', 50822) - "GET /static/example-cropped1.jpg HTTP/1.1" 200
INFO: ('82.28.48.184', 50815) - "GET /static/example-cropped3.jpg HTTP/1.1" 200
INFO: ('82.28.48.184', 50824) - "GET /static/example-cropped4.jpg HTTP/1.1" 200
INFO: ('82.28.48.184', 50824) - "GET /favicon.ico HTTP/1.1" 404
INFO: ('82.28.48.184', 50873) - "POST /detect HTTP/1.1" 200
INFO

{'cropped_items': ['../static/tmp/caae67bd-763d-4b6c-a15a-8dda26c9bc13.jpg', '../static/tmp/ff9a2ffc-45e9-4570-a696-572c0c0f1894.jpg', '../static/tmp/4d21239d-fde2-4e09-ae00-5845fde1d1e9.jpg', '../static/tmp/c37faef2-a352-4e93-8b92-4f8e66372f43.jpg', '../static/tmp/635b3464-7700-4f37-b9e3-ee7c4dbc90a2.jpg', '../static/tmp/507f947b-9d21-4b39-9e6a-f1d301fd4110.jpg'], 'categories': [2, 2, 2, 4, 7, 8]}


INFO: ('82.28.48.184', 50879) - "GET /static/tmp/635b3464-7700-4f37-b9e3-ee7c4dbc90a2.jpg HTTP/1.1" 200
INFO: ('130.159.236.1', 52138) - "GET / HTTP/1.1" 200
INFO: ('130.159.236.1', 52138) - "GET /static/style.css HTTP/1.1" 304
INFO: ('130.159.236.1', 52142) - "GET /static/example.jpg HTTP/1.1" 304
INFO: ('130.159.236.1', 52138) - "GET /static/client.js HTTP/1.1" 304
INFO: ('130.159.236.1', 52142) - "GET /static/scroll-reveal-conf.js HTTP/1.1" 304
INFO: ('130.159.236.1', 52144) - "GET /static/example-cropped1.jpg HTTP/1.1" 304
INFO: ('130.159.236.1', 52146) - "GET /static/example-cropped2.jpg HTTP/1.1" 304
INFO: ('130.159.236.1', 52138) - "GET /static/example-cropped3.jpg HTTP/1.1" 304
INFO: ('130.159.236.1', 52148) - "GET /static/example-cropped4.jpg HTTP/1.1" 304
INFO: 3.262: POST https://background-removal.rorocloud.io/predict
INFO: ('82.28.48.184', 50891) - "POST /analyze HTTP/1.1" 200
INFO: ('82.28.48.184', 50891) - "GET /static/image_data/e840fedbcd2f9993452cc8fa0f366b6d.jpg HTTP

['../static/image_data/e840fedbcd2f9993452cc8fa0f366b6d.jpg', '../static/image_data/vice-high-waisted-slash-knee-skinny-jeans-black.jpeg']
